### Базовое решение кейса "Рекомендательная система видео" 
### Кейсодержатель: RUTUBE
#### Описание решения: 

Приведенное базовое решение основано на подсчете совстречаемостей видео в сессии пользователя, или если говорить "по-бытовому": посчитаем как часто один товар попадает вместе с другим в одну корзину. На рисунке ниже вы можете видеть, что к пиву очень разумно рекомендовать рыбку, ведь ее частенько берут в дополенение к этому напитку. 

Однако вполне реальной выглядит ситуация, что пиво встречается чаще в корзине с другим продуктом - с молоком. Почему так происходит? Молоко много с чем часто встречается в корзине просто потому что это очень популярный товар в магазине, в отличие от вяленой рыбки. Но было бы странно получать в рекомендациях к пиву молоко. Поэтому для расчета схожести применяется нормировка на популярность товаров! Далее на основе полученных для каждой пары товаров чисел, рекомендации можно отранжировать и получить в выдаче красоту, а не молоко:)

Подробнее читайте тут: https://www.inf.unibz.it/~ricci/ISR/papers/p293-davidson.pdf

![Рекомендации к пиву](fish_to_beer.jpg)

In [1]:
import pandas as pd
import os
import gc

print("loading data ...")
hack = pd.read_csv("small_player_starts_train.csv")

loading data ...


In [3]:
hack.head(5)

,date,user_id,item_id,watch_time,is_autorized
0,2023-08-21 15:53:30+03:00,user_7941459,video_1535803,22,0
1,2023-08-21 16:17:58+03:00,user_17893165,video_796847,374,0
2,2023-08-21 21:18:12+03:00,user_25219604,video_1905049,11,0
3,2023-08-21 01:06:10+03:00,user_24477183,video_1156618,1,0
4,2023-08-21 17:24:32+03:00,user_25483180,video_500499,20,0


#### Для базового решения возьмем данные о просмотрах пользователей

In [4]:
print("Amount of events: ", len(hack))
users = hack.user_id.unique()
print("Unique users: ", len(users))
print("Unique items: ", hack.item_id.nunique())
print("Min date: ", hack.date.min())
print("Max date: ", hack.date.max())

Amount of events:  2439644
Unique users:  1313891
Unique items:  261288
Min date:  2023-08-21 00:00:00+03:00
Max date:  2023-08-21 23:59:59+03:00


In [5]:
# preprocess date
hack['date_short'] = hack.date.apply(lambda l: l.split(" ")[0])

#### Отфильтруем только "активных" пользователей, то есть тех, у кого более 1 просмотра за сессию (в нашем случае - за сутки).

In [6]:
%%time
groupby = hack.groupby(["user_id", "date_short"]).count()
users_active = groupby[groupby['item_id'] >= 2].index.unique()

CPU times: total: 4.48 s
Wall time: 4.49 s


In [7]:
users_active = set(users_active)

In [8]:
from tqdm import tqdm 
tqdm.pandas()
hack['active'] = hack.progress_apply(lambda l: True if (l.user_id, l.date_short) in users_active else False, axis=1)
hack=hack[hack.active]

100%|██████████| 2439644/2439644 [00:28<00:00, 84655.68it/s]


In [9]:
print("Финальное число событий", len(hack))

Финальное число событий 1457176


In [10]:
print("Финальное число видео", hack.item_id.nunique())

Финальное число видео 164056


#### Построим простую модель рекомендаций, основанную на совместной встречаемости видео в сессиях пользователей - "Вместе с этим товаром часто покупают.."

В качеcтве сессии будем брать просмотры пользователя за сутки

In [11]:
def calc_pairs(train):
    print("Get sessions")
    count = train.groupby(['item_id']).size().rename('count_source').reset_index()[
        ['item_id', 'count_source']]
    dt = train.groupby(['user_id', 'date_short'])['item_id'].agg(list).rename('pair').reset_index()
    df = train[['user_id', 'date_short', 'item_id']].merge(dt, on=['user_id', 'date_short'], how='left')
    
    del dt
    gc.collect()

    # Explode the rows vs list of articles
    print("Explode the rows vs list of articles")
    df = df[['item_id', 'pair']].explode(column='pair')
    gc.collect()
    
    print("Discard duplicates")
    df = df.loc[df['item_id'] != df['pair']].reset_index(drop=True)
    
    print("Count how many times each pair combination happens")
    df = df.groupby(['item_id', 'pair']).size().rename('count').reset_index()
    
    print("join with frecuency of source item")
    df = df.merge(count, on=['item_id'], how='left')
    count = count.rename({'count_source': 'count_target', 'item_id': 'pair'}, axis=1)
    df = df.merge(count, on=['pair'], how='left')

    df['score'] = df.apply(lambda l: l['count'] / (l['count_source']*l['count_target']), axis=1)
    df.sort_values(by=['score'], ascending=False, inplace=True)
    gc.collect()
    return df

In [12]:
%%time
rules = calc_pairs(hack[['user_id', 'item_id', 'date_short']])

Get sessions
Explode the rows vs list of articles
Discard duplicates
Count how many times each pair combination happens
join with frecuency of source item
CPU times: total: 2min 51s
Wall time: 2min 51s


In [13]:
rules.to_csv("test_random_sample_2w_clear.csv", index=False)

In [14]:
rules = pd.read_csv("test_random_sample_2w_clear.csv")

In [15]:
rules.item_id.nunique()

157125

In [16]:
# проверим результаты
features = pd.read_parquet("videos.parquet")

ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.

#### Посмотрим на рекомендации к смешарикам

In [ ]:
# посмотрим, что порекомендует модель, если передадим ей смешариков

features[features.item_id == 'video_2107999']

,item_id,video_title,author_title,tv_title,season,video_description,category_title,publicated,duration,channel_sub,tv_sub,ctr.CTR_10days_21_07,ctr.CTR_10days_01_08,ctr.CTR_10days_10_08,ctr.CTR_10days_21_08
2267968,video_2107999,"Смешарики 5, 45 серия",Рики,Смешарики 2D 5 сезон,5,None,Детям,2022-12-01 11:49:03+03:00,405000,1638,0,0.117647,0.25,0.076923,0.176471


In [ ]:
predictions = rules[rules.item_id=='video_2107999'].pair.to_list()[:10]

In [ ]:
prediction_names = [features[features.item_id == pred]['video_title'].values[0] for pred in predictions]
prediction_names

['Toyota Rav4 установка музыки, процессорная по канальная в штатные места.',
 'ПОЧЕМУ ВАЖНО ПЕРВОЕ ВПЕЧАТЛЕНИЕ?',
 'прохождение зе Твинс инди хоррор',
 'Смешарики 5, 39 серия',
 'Смешарики 5, 47 серия',
 'ДОМ 2: Драка Сергея Пынзаря и Дарьи Черных',
 'Смешарики 5, 48 серия',
 'Смешарики 5, 41 серия',
 'ФАНФИК: Т/И В ГРУППЕ С БТС И БЛЭКПИНК? ЧАСТЬ 12 #bts  #army #арми #фф #фанфик #blackpink #pov',
 'Смешарики 5, 40 серия']

#### Теперь на рекомендации к спортивному матчу "Зенит - Урал"

In [ ]:
# посмотрим, что порекомендует модель, если передадим ей футбольный матч

features[features.item_id == 'video_987926']

,item_id,video_title,author_title,tv_title,season,video_description,category_title,publicated,duration,channel_sub,tv_sub,ctr.CTR_10days_21_07,ctr.CTR_10days_01_08,ctr.CTR_10days_10_08,ctr.CTR_10days_21_08
1293938,video_987926,Зенит - Урал. Обзор матча Мир РПЛ 01.04.2023,МАТЧ!,Обзоры матчей,0,📺🔥 Смотри лучший спортивный контент в одном ме...,Спорт,2023-04-01 21:19:55+03:00,487489,31571,0,0.153846,0.0,0.022727,0.0


In [ ]:
predictions = rules[rules.item_id=='video_987926'].pair.to_list()[:10]
prediction_names = [features[features.item_id == pred]['video_title'].values[0] for pred in predictions]
prediction_names

['Зенит - Динамо. Обзор матча Мир РПЛ 06.08.2023']

#### Посмотрим, что порекомендует модель, если передадим ей стенд ап

In [ ]:
features[features.item_id == 'video_1496562']

,item_id,video_title,author_title,tv_title,season,video_description,category_title,publicated,duration,channel_sub,tv_sub,ctr.CTR_10days_21_07,ctr.CTR_10days_01_08,ctr.CTR_10days_10_08,ctr.CTR_10days_21_08
1376255,video_1496562,"STAND UP, 9 сезон, 1 выпуск",Телеканал ТНТ,STAND UP,9,Премьера! На ТНТ стартует уже девятый сезон шо...,Телепередачи,2021-09-02 18:55:55+03:00,2707000,75496,0,0.066406,0.143548,0.110927,0.117761


In [ ]:
predictions = rules[rules.item_id=='video_1496562'].pair.to_list()[:10]
prediction_names = [features[features.item_id == pred]['video_title'].values[0] for pred in predictions]
prediction_names

['Страна талантов, 2 сезон, 1 выпуск',
 'STAND UP, 9 сезон, 12 выпуск',
 'STAND UP, 8 сезон, 13 серия',
 'Mary Gu vs Акула (Оксана Почепа) | Битва Поколений | 8 ВЫПУСК',
 '"StandUp": премьерный выпуск нового сезона',
 'STAND UP, 3 сезон, 1 выпуск',
 'STAND UP, 1 сезон, 2 выпуск (эфир 29.09.2013)',
 'STAND UP, 4 сезон, 5 выпуск',
 'Stand Up: Новый сезон',
 'STAND UP, 4 сезон, 4 выпуск']

#### Cделаем предсказания для тестовых пользователей, будем притягивать рекомендации к последнему видео, которое пользователь смотрел на трейне

Для пользователей, которые не попали в трейн, но есть на тесте, порекомендуем самые популярные видео из трейна

In [ ]:
most_popular = list(hack.item_id.value_counts()[:10].to_frame().index)

In [ ]:
features[features.item_id.isin(most_popular)][:5]

,item_id,video_title,author_title,tv_title,season,video_description,category_title,publicated,duration,channel_sub,tv_sub,ctr.CTR_10days_21_07,ctr.CTR_10days_01_08,ctr.CTR_10days_10_08,ctr.CTR_10days_21_08
71943,video_2171643,Денис Власенко о школьном буллинге,Вокруг•ТВ,None,0,Денис Власенко дал эксклюзивное интервью «Вокр...,Лайфстайл,2023-08-17 14:36:37+03:00,61280,2146,0,NaN,NaN,NaN,0.000000
155423,video_68646,"Выжить в Дубае, 8 выпуск",Телеканал ТНТ,Выжить в Дубае,1,Премьера! «Выжить в Дубае» – новое масштабное ...,Телепередачи,2023-08-13 21:00:12+03:00,5208840,75496,0,NaN,NaN,NaN,0.411112
406823,video_1594159,"Выжить в Дубае, 7 выпуск",Телеканал ТНТ,Выжить в Дубае,1,Премьера! «Выжить в Дубае» – новое масштабное ...,Телепередачи,2023-08-06 21:00:18+03:00,5930640,75496,0,NaN,NaN,0.448765,0.319222
956478,video_283933,"Выжить в Дубае, 9 выпуск",Телеканал ТНТ,Выжить в Дубае,1,Премьера! «Выжить в Дубае» – новое масштабное ...,Телепередачи,2023-08-20 21:00:23+03:00,5616280,75496,0,NaN,NaN,NaN,0.572971
1123138,video_830290,На разок. Мужское / Женское. Выпуск от 18.08.2023,Первый канал,Мужское / Женское,0,Ангелина Перминова выросла в детском доме и се...,Телепередачи,2023-08-18 17:59:14+03:00,2437056,45848,0,NaN,NaN,NaN,0.155233


#### Загружаем пример submission'a

In [ ]:
submission = pd.read_csv("sample_submission.csv")

In [ ]:
submission

,user_id,recs
0,user_26511551,"['video_0', 'video_0', 'video_0', 'video_0', '..."
1,user_29194819,"['video_0', 'video_0', 'video_0', 'video_0', '..."
2,user_29734049,"['video_0', 'video_0', 'video_0', 'video_0', '..."
3,user_955460,"['video_0', 'video_0', 'video_0', 'video_0', '..."
4,user_7065521,"['video_0', 'video_0', 'video_0', 'video_0', '..."
...,...,...
97235,user_29281681,"['video_0', 'video_0', 'video_0', 'video_0', '..."
97236,user_3912848,"['video_0', 'video_0', 'video_0', 'video_0', '..."
97237,user_28389099,"['video_0', 'video_0', 'video_0', 'video_0', '..."
97238,user_18951296,"['video_0', 'video_0', 'video_0', 'video_0', '..."


#### Находим последнее просмотренное видео пользователями из теста (которые также есть и в трейне)

In [ ]:
hack = hack[hack.user_id.isin(submission['user_id'].unique())].sort_values(by=['date'])

In [ ]:
%%time
last_watched_videos = (
    hack[['user_id', 'item_id']]
    .groupby(['user_id'], as_index=False)
    .agg('last')
)

CPU times: user 218 ms, sys: 0 ns, total: 218 ms
Wall time: 215 ms


In [ ]:
last_watched_videos = last_watched_videos.set_index('user_id').to_dict()['item_id']

In [ ]:
items_in_train = {source: list(group.index) for source, group in rules.groupby(['item_id'])}

In [ ]:
from tqdm import tqdm 
tqdm.pandas()


def get_recs_for_users(rules, sample_submission, last_watched_videos, items_in_train, num_candidates: int=10):
    
    for i, row in tqdm(sample_submission.iterrows()):
        if row.user_id in last_watched_videos:
            video = last_watched_videos[row.user_id]
            video_ids = items_in_train[video] if video in items_in_train else ''
            if len(video_ids)>0:
                sample_submission.loc[i, 'recs'] = rules.iloc[video_ids].item_id.to_list()[:num_candidates]
            else:
                sample_submission.loc[i, 'recs'] = most_popular
        else:
            sample_submission.loc[i, 'recs'] = most_popular
    return sample_submission

In [ ]:
# получаем предсказания
submission = get_recs_for_users(rules, submission, last_watched_videos, items_in_train)

97240it [00:30, 3143.90it/s]


In [ ]:
submission

,user_id,recs
0,user_26511551,[video_1580070]
1,user_29194819,"[video_283933, video_68646, video_1508623, vid..."
2,user_29734049,"[video_283933, video_68646, video_1508623, vid..."
3,user_955460,"[video_283933, video_68646, video_1508623, vid..."
4,user_7065521,"[video_5763, video_5763, video_5763, video_576..."
...,...,...
97235,user_29281681,"[video_283933, video_68646, video_1508623, vid..."
97236,user_3912848,"[video_1200669, video_1200669, video_1200669, ..."
97237,user_28389099,"[video_144691, video_144691, video_144691, vid..."
97238,user_18951296,"[video_283933, video_68646, video_1508623, vid..."


In [ ]:
submission.to_csv("submission.csv", index=False)